In [1]:
import bs4
import requests
import html5lib

In [2]:
base_url = "http://www.ilga.gov"

In [3]:
leg_url = base_url + "/legislation/"

In [4]:
print(leg_url)

http://www.ilga.gov/legislation/


In [5]:
r = requests.get(leg_url)

In [6]:
html_text = r.text.encode('iso-8859-1')

In [7]:
soup = bs4.BeautifulSoup(html_text, "html5lib")

In [8]:
links = soup.find_all("a")

In [9]:
mid_links = []
for a in links:
    if a.has_attr("href") and a['href'].startswith('grplist.asp'):
        mid_url = leg_url + a['href']
        mid_r = requests.get(mid_url)
        mid_html_text = mid_r.text.encode('iso-8859-1')
        mid_soup = bs4.BeautifulSoup(mid_html_text, "html5lib")
        mid_links.extend(mid_soup.find_all("a"))
        

In [10]:
sbill_links = []
hbill_links = []
for mid_a in mid_links:
    if mid_a.text.startswith("SB"):
        bill_url = base_url + mid_a['href']
        sbill_links.append(bill_url)
    elif mid_a.text.startswith("HB"):
        bill_url = base_url + mid_a['href']
        hbill_links.append(bill_url)

In [11]:
len(sbill_links)

3896

In [12]:
len(hbill_links)

5668

In [13]:
r_test = requests.get(sbill_links[0])
html_test = r_test.text.encode('iso-8859-1')
soup_test = bs4.BeautifulSoup(html_test, "html5lib")

In [15]:
bill_info = {}
soup_test.find_all("span")

[<span class="close">×</span>,
 <span style="white-space: nowrap"><a class="goog-logo-link" href="https://translate.google.com" target="_blank"><img alt="Google Translate" height="14" src="https://www.gstatic.com/images/branding/googlelogo/1x/googlelogo_color_42x16dp.png" style="padding-right: 3px;" width="37"/>Translate</a></span>,
 <span class="heading">Bill Status of SB0001</span>,
 <span class="content"><i>101st General Assembly</i></span>,
 <span class="heading2">Short Description:</span>,
 <span class="content">MINIMUM WAGE/INCOME TAX CREDIT</span>,
 <span class="heading2">Senate Sponsors</span>,
 <span class="content">Sen. </span>,
 <span class="content"> and </span>,
 <span class="heading2">House Sponsors</span>,
 <span class="content">(Rep. </span>,
 <span class="content"> and </span>,
 <span class="heading2">Last Action</span>,
 <span class="heading2">Statutes Amended In Order of Appearance</span>,
 <span class="heading2">Synopsis As Introduced</span>,
 <span class="content n